# Test 6

should be answered by building and calibrating a 10-period Black-Derman-Toy model for the short-rate.

Period 1 2 3 4 5 6 7 8 9 10

Spot Rate 3.0% 3.1% 3.2% 3.3% 3.4% 3.5% 3.55% 3.6% 3.65% 3.7%

Assume that $Z^6_0 = \frac{100}{(1+r_6)^6}$


In [1]:
SPOT_RATES = [.03, .031, .032, .033, .034, .035, .0355, .036, .0365, .037]
TEST_RATES = [.073, .0762, .081, .0845, .092, .0964, .1012, .1045, .1122, .1155, .1192, .122, .1232]

In [2]:
from utils import *

In [3]:
bdt = BlackDermanToy(SPOT_RATES, b=0.05)
a = bdt.calibrate()

In [4]:
rates, ep = bdt.calculate_spot_rates()
print(bdt.verify_correctness())

True


Once your model has been calibrated, compute the price of a payer swaption with notional $1M$ that expires at time $t=3$ with an option strike of $0$. You may assume the underlying swap has a fixed rate of $3.9\%$ and that if the option is exercised then cash-flows take place at times $t=4,…,10$. (The cash-flow at time $t=i$ is based on the short-rate that prevailed in the previous period, i.e. the payments of the underlying swap are made in arrears.)

In [5]:
""" Using short rates and knowledge that this is a payer swaption, 
    we can calculate the value of the swap until time t=4 
    Option owner doesn't receive/make coupon payments, from there we do regular option
    lattice reduction"""
fixed_rate = 0.039
short_rate_lattice = bdt.short_rates

In [6]:
swap_lattice = reduce_swap(fixed_rate, short_rate_lattice, start=0, periods=9, qu=.5, qd=.5, debug=False)
values = [[max(x, 0)for x in swap_lattice[3]]]

call = reduce_call(values, swap_lattice, short_rate_lattice, n=3, K=0, qu=.5, qd=.5, american=False, debug=False)

In [7]:
print(f"swap value is ${round(call[0][0]*1_000_000, 2)}")

swap value is $4085.17


# Q2

In [9]:
bdt = BlackDermanToy(SPOT_RATES, b=0.1)
bdt.calibrate()
rates, ep = bdt.calculate_spot_rates()
short_rate_lattice = bdt.short_rates

swap_lattice = reduce_swap(fixed_rate, short_rate_lattice, start=0, periods=9, qu=.5, qd=.5, debug=False)
values = [[max(x, 0)for x in swap_lattice[3]]]

call = reduce_call(values, swap_lattice, short_rate_lattice, n=3, K=0, qu=.5, qd=.5, american=False, debug=False)

print(f"swap value is ${round(call[0][0]*1_000_000, 2)}")

swap value is $8027.14


# Q3
assume 1-step hazard rate  is $h_{ij}=ab^{j-\frac{i}{2}}$
Compute the price of a zero-coupon bond with face value 
$F=100$ and recovery $R=20\%$.

In [31]:
n = 10
a = .01
b = 1.01
F = 100   # face value
R = .2    # recovery

srl = generate_short_rate_lattice(r00=.05 , n=n, u=1.1, d=0.9)

def h(i, j, a=a, b=b):
    """Get the hazard rate given some i and j"""
    return a*b**(j-i/2)

start out with the face value. then start reducing using pricing model for defaultable ZCB with recovery

get all the none default values ($\eta = 0$) since default values ($\eta = 1$)

In [41]:

def price_defaultable_zcb(face_value, periods, srl, R, q_u=0.5, q_d=0.5):
    zcb_matrix = [[100]*(periods + 1)]
    
    for i in range(n)[::-1]:
        last_prices = zcb_matrix[0]
        new_prices = []

        for j in range(i + 1):
            disc = 1/(1+srl[i][j])
            hij = h(i,j)   # the hazard rate at time i state j
            z_ij = disc*(q_u*(1-hij)*last_prices[j + 1] +
                         q_d*(1-hij)*last_prices[j] +
                         q_u*(hij)*R +
                         q_d*(hij)*R
                        )
            new_prices.append(z_ij)

        zcb_matrix.insert(0, new_prices)
    print(zcb_matrix)
    print("zcb length: ", len(zcb_matrix))
    return zcb_matrix[0][0]

print(round(price_defaultable_zcb(100, 10, srl, R*100), 2))

length of previous prices 11
i:  9
hij 0.00956211181821209 for i: 9 j: 0
last price at 1 and 0
hij 0.009657732936394211 for i: 9 j: 1
last price at 2 and 1
hij 0.009754310265758152 for i: 9 j: 2
last price at 3 and 2
hij 0.009851853368415734 for i: 9 j: 3
last price at 4 and 3
hij 0.009950371902099893 for i: 9 j: 4
last price at 5 and 4
hij 0.010049875621120889 for i: 9 j: 5
last price at 6 and 5
hij 0.010150374377332098 for i: 9 j: 6
last price at 7 and 6
hij 0.01025187812110542 for i: 9 j: 7
last price at 8 and 7
hij 0.010354396902316473 for i: 9 j: 8
last price at 9 and 8
hij 0.01045794087133964 for i: 9 j: 9
last price at 10 and 9
length of previous prices 10
i:  8
hij 0.009609803444828162 for i: 8 j: 0
last price at 1 and 0
hij 0.009705901479276444 for i: 8 j: 1
last price at 2 and 1
hij 0.009802960494069209 for i: 8 j: 2
last price at 3 and 2
hij 0.009900990099009901 for i: 8 j: 3
last price at 4 and 3
hij 0.01 for i: 8 j: 4
last price at 5 and 4
hij 0.0101 for i: 8 j: 5
last pri

# Q4 

Solve for hazard rates given some markets bond prices. Use same hazard rate for every period to take advantage of risk neutral dynamics of interest rates. meaning discount for some period $k$ is $Z_0^{t_k}=d(0,t_n)$

In [7]:
F = 100
r = 0.05

In [31]:
# actual values
bond_prices = [100.92, 91.56, 105.6, 98.90, 137.48]
coupons = [.05, .02, .05, .05, .1]
recovery = [.1, .25, .5, .1, .2]

In [32]:
# example values
test_bond_prices = [101.22, 92.58, 107.44, 104.05, 145.82]
test_coupons = [.05, .02, .05, .05, .1]
test_recovery = [.1, .25, .5, .1, .2]

In [35]:
# months = 12*years
# time = [i for i in range(0, months+1, 6)]
survival_prob = [1.0]
discount_rates = [1.0]

In [45]:
from sympy.solvers import solve
from sympy import *
from scipy.optimize import minimize
import numpy as np

def calibrate_hazard_rate(true_prices, coupon_rates, recovery_rates, interest_rate=0.05, F=100,):
    """ Returns hazard rates calibrated from true_prices of 
    Defaultable bonds with recovery. Assuming that the hazard rate is stable
    through out the year.
    """
    t = 0
    hazards = []
    survival_prob = [1]
    default_prob = [None]
    
    # discount rates for each semi annual period
    d = [round(1/(1+interest_rate/2)**i, 3) for i in range(1, len(true_prices) * 2 + 1)]
    print("discount ", d)

    for t_price, coupon_rate, recovery_rate in zip(true_prices, coupon_rates, recovery_rates):
        if t == 2:
            break
        print(f"at year {t}")
        # get potential coupon for each time period
        coupons = [coupon_rate*F for _ in range((t+1)*2)]
        coupons[-1] += F
        
        # get potential recovery value
        # recovery = [recovery_rate*F for _ in range((t+1)*2)]
        recovery = recovery_rate*F
        
        print("recovery ", recovery)
        print("coupons: ", coupons)
        
        # solving for the t-th hazard using all previous hazard rates calced
        # pre calculate using previous rates
        known = 0
        for i in range(t):
            print(f"calc year {i}")
            q1 = survival_prob[i*2+1]
            q2 = survival_prob[(i+1)*2]
            print("q1, q2: ", q1, q2)
            print("d1, d2: ", d[i*2], d[i*2+1])

            op1 = d[i*2]*(q1*coupons[i*2] + (1-q1)*recovery)
            op2 = d[i*2+1]*(q2*coupons[i*2+1] + (1-q2)*recovery)

            known += op1 + op2
            print(f"disc expected val of first half in year {i}: {op1}")
            print(f"disc expected val of second half in year {i}: {op2}")
            
        
        # create equations to solve
        print(f"solve in year {t}")
        print(f"survival_prob: {survival_prob[-1]}")
        h = symbols('h')    # hazard rate for both semi periods
        
        q1 = survival_prob[-1]*(1-h)   # survival prob for first half
        q2 = q1*(1-h)                  # survival prob for second half

        p1 = d[t*2]*(q1*coupons[t*2] + (1-q1)*recovery)   # discounted expected value in first half
        p2 = d[t*2+1]*(q2*coupons[t*2+1] + (1-q2)*recovery)   # discounted expected value in second half
        print(f"q1 {q1}\tp1 {p1}")
        print(f"q2 {q2}\tp2 {p2}")
        
        error = (known + p1 + p2 - t_price)**2

        def f(x):
            # TODO check that its greater than the last one
            res = error.subs(h, x[0]/100)
            return np.float64(res)

        res = minimize(f, 2, method='BFGS')
        if len(res.x) > 1:
            raise ValueError("we have more than two possible solutions: ", res)

        h_r = res.x[0]/100
        
        survival_prob.append(q1.subs(h, h_r))
        survival_prob.append(q2.subs(h, h_r))
        print(survival_prob)
        
        t += 1
        print()



calibrate_hazard_rate(test_bond_prices, test_coupons, test_recovery)
    

discount  [0.976, 0.952, 0.929, 0.906, 0.884, 0.862, 0.841, 0.821, 0.801, 0.781]
at year 0
recovery  10.0
coupons:  [5.0, 105.0]
solve in year 0
survival_prob: 1
q1 1 - h	p1 4.88*h + 4.88
q2 (1 - h)**2	p2 90.44*(1 - h)**2 + 9.52
[1, 0.979209705410111, 0.958851647169356]

at year 1
recovery  25.0
coupons:  [2.0, 2.0, 2.0, 102.0]
calc year 0
q1, q2:  0.979209705410111 0.958851647169356
d1, d2:  0.976 0.952
disc expected val of first half in year 0: 2.41870053295383
disc expected val of second half in year 0: 2.80498433357978
solve in year 1
survival_prob: 0.958851647169356
q1 0.958851647169356 - 0.958851647169356*h	p1 20.4877831450676*h + 2.73721685493237
q2 (0.958851647169356 - 0.958851647169356*h)*(1 - h)	p2 69.762*(0.958851647169356 - 0.958851647169356*h)*(1 - h) + 22.65
[1, 0.979209705410111, 0.958851647169356, 0.916065356244352, 0.875188293609796]



In [43]:
        
def calibrate_hazard_rate_(true_prices, coupons, recovery, F=100,):
    """test this out with a single bond first. should work in theory.
    if it does, you can kinda bootstrap the process for other bonds.
    """
    true_price = true_prices[0]
    coupons = [c*F for c in coupons]
    c1, c2 = coupons[0], coupons[0] + F
    recovery_val = recovery[0]*F
    
    d1, d2 = 1/(1.025), 1/(1.025)**2
    h0 = symbols('h0')    # hazard rates
    
    q0 = 1
    q1 = q0*(1-h0)
    q2 = q1*(1-h0)
    print(c1, c2)
    print("d1", d1)
    print("d2", d2)

    p1 = d1*(q1*c1 + (1-q1)*recovery_val)
    p2 = d2*(q2*c2 +(1-q2)*recovery_val)
    
    print("price1: ", p1)
    print("price2: ", p2)
    
    eq = (p1 + p2 - true_price)**2
    
    print(eq)
    
    def f(x):
        res = eq.subs(h0, x[0]/100)
        print(res)
        return np.float64(res)

    print(minimize(f, 2, method='BFGS'))
    